In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, TerminationCondition
import psycopg2
import pandas as pd

## Conectar Base de datos y Preparacion de los datos a utilizar

In [2]:
conexion = psycopg2.connect(
        dbname="buap",
        user="postgres",
        password="contrasena",
        host="localhost",
        port="5432"
    )
cursor = conexion.cursor()
datos = {}

In [3]:
# Profesores (con nombre)
cursor.execute("SELECT id_profesor, nombre FROM Profesor LIMIT 30")
profesores = cursor.fetchall()
datos['Id_Profesores'] = [row[0] for row in profesores] # Lista de IDs de profesores
datos['nombres_profesor'] = {row[0]: row[1] for row in profesores} # Diccionario de nombres de profesores


In [4]:
datos['nombres_profesor']

{100002999: 'GARCIA - ZENTENO EDUARDO',
 100004288: 'AYALA - ROJAS JUAN LEONARDO',
 100012622: 'CRUZ - ALMANZA GRACIANO',
 100014033: 'MORENO - BARBOSA FERNANDO',
 100020344: 'GARCIA - SALAZAR RUBEN',
 100054211: 'SALDAÑA - SALDAÑA XOCHITL INES',
 100065344: 'GARCIA - SALGADO GODOFREDO',
 100104677: 'RUIZ - TENORIO JOSE ANTONIO',
 100104866: 'RAMIREZ - MENDOZA MARCELINO',
 100124799: 'VAZQUEZ - ESPINOSA DE LOS MONTEROS MARIA GUADALUPE',
 100145955: 'HERNANDEZ - DE LA LUZ JOSE ALVARO',
 100171599: 'JUAREZ - RAMON DIONICIO',
 100178366: 'LEMBRINO - PEREZ IMELDA LUZ',
 100180788: 'MEJIA - GOMEZ MARIA ROSA',
 100209699: 'HERNANDEZ - VALDES IBET',
 100210066: 'NORIEGA - TOVILLA NANCY ALEJANDRA',
 100210100: 'RUIZ - ARENAS JOSE MIGUEL ANGEL',
 100226555: 'ALONSO - PEREZ CARLOS',
 100233866: 'ARGUELLO - MORALES MARCOS',
 100238800: 'CONTRERAS - JUAREZ ROBERTO',
 100239655: 'ROSAS - LORANCA MARIA GABRIELA',
 100240399: 'ROBLEDO - ROBLEDO RAFAEL',
 100253488: 'MORIN - CASTILLO MARIA MONSERRAT',

In [5]:
# Materias (con nombre)
cursor.execute("SELECT id_materia, nombre, horas_por_semana, id_tipo_clase FROM Materia LIMIT 30")
materias = cursor.fetchall()
datos['id_Materias'] = [row[0] for row in materias] # Lista de IDs de materias
datos['nombres_materia'] = {row[0]: row[1] for row in materias} # Diccionario de nombres de materias
datos['Horas_Semana_Materia'] = {row[0]: row[2] for row in materias} # Diccionario de horas por semana de cada materia
datos['Tipo_Clase'] = {row[0]: row[3] for row in materias} # Diccionario de tipo de clase de cada materia

In [6]:
# Aulas
cursor.execute("SELECT id_aula, id_edificio, id_tipo_clase FROM Aula LIMIT 30")
rows = cursor.fetchall()

datos['Aulas'] = [(row[0], row[1]) for row in rows] # Lista de IDs compuestos (id_aula, id_edificio)
datos['Tipo_Aula'] = {(row[0], row[1]): row[2] for row in rows}# Diccionario: clave compuesta (id_aula, id_edificio) -> tipo clase


# Bloques horarios
datos['Dia'] = list(range(1, 6)) # Lunes a Viernes
datos['Hora'] = list(range(1, 11)) # 10 bloques horarios por día/ de 8 am a 6 pm

In [7]:
cursor.execute("""
    SELECT A.id_aula, A.id_edificio, M.id_materia,
           CASE WHEN A.id_tipo_clase = M.id_tipo_clase THEN 1 ELSE 0 END AS compatible,
           M.id_tipo_clase AS tipo_materia,
           A.id_tipo_clase AS tipo_aula
    FROM Aula A
    CROSS JOIN Materia M
    WHERE A.id_aula = ANY(%s) AND M.id_materia = ANY(%s)
""", ([a[0] for a in datos['Aulas']], datos['id_Materias']))  # solo ids de aula

datos['Compatibiliadad_Aula_Materia'] = {
    ((row[0], row[1]), row[2]): {
        'compatible': row[3],
        'tipo_materia': row[4],
        'tipo_aula': row[5]
    }
    for row in cursor.fetchall()
}


In [8]:
from collections import defaultdict

# Crear diccionarios como defaultdict
datos['Materias_por_Profesor'] = defaultdict(list)
datos['Profesores_por_Materia'] = defaultdict(list)

# Obtener relaciones profesor-materia desde la tabla 'profesor_materia'
cursor.execute("""
    SELECT id_profesor, id_materia
    FROM profesor_materia
    WHERE id_profesor = ANY(%s) AND id_materia = ANY(%s)
""", (datos['Id_Profesores'], datos['id_Materias']))

asignaciones = cursor.fetchall()

# Cargar relaciones reales
for profe_id, materia_id in asignaciones:
    datos['Materias_por_Profesor'][profe_id].append(materia_id)
    datos['Profesores_por_Materia'][materia_id].append(profe_id)

# Asegurar que todas las claves estén presentes, incluso si no tienen asignaciones
for profe_id in datos['Id_Profesores']:
    if profe_id not in datos['Materias_por_Profesor']:
        datos['Materias_por_Profesor'][profe_id] = []  # asegura existencia de la clave

for materia_id in datos['id_Materias']:
    if materia_id not in datos['Profesores_por_Materia']:
        datos['Profesores_por_Materia'][materia_id] = []  # asegura existencia de la clave


In [9]:
datos['Profesores_por_Materia'], datos['Materias_por_Profesor']

(defaultdict(list,
             {1: [100004288, 100020344],
              2: [100004288, 100104866],
              3: [100004288, 100226555],
              7: [100104677],
              4: [100104866, 100226555],
              5: [100226555],
              6: [100226555],
              16: [100233866],
              21: [100233866],
              19: [100239655],
              17: [100267677],
              8: [],
              9: [],
              10: [],
              11: [],
              12: [],
              13: [],
              14: [],
              15: [],
              18: [],
              20: [],
              22: [],
              23: [],
              24: [],
              25: [],
              26: [],
              27: [],
              28: [],
              29: [],
              30: []}),
 defaultdict(list,
             {100004288: [1, 2, 3],
              100020344: [1],
              100104677: [7],
              100104866: [2, 4],
              100226555: [3, 4, 5, 6]

## Crear Modelo de Pyomo

In [10]:
# Definir los conjuntos
model = pyo.ConcreteModel()

# Conjuntos
model.Profesores = pyo.Set(initialize=datos['Id_Profesores'])
model.Materias = pyo.Set(initialize=datos['id_Materias'])
model.Aulas = pyo.Set(initialize=[(aula_id, edificio_id) for aula_id, edificio_id in datos['Aulas']])
model.BloquesHorarios = pyo.Set(initialize=[(dia, hora) for dia in datos['Dia'] for hora in datos['Hora']])


# Parámetros
model.Horas_Semana_Materia = pyo.Param(model.Materias, initialize=datos['Horas_Semana_Materia'])
model.Tipo_Clase = pyo.Param(model.Materias, initialize=datos['Tipo_Clase'])
model.Tipo_Aula = pyo.Param(model.Aulas, initialize=datos['Tipo_Aula'])
model.Compatibilidad_Aula_Materia = pyo.Param(model.Aulas, model.Materias, initialize={
    (aula, materia): datos['Compatibiliadad_Aula_Materia'].get((aula, materia), {}).get('compatible', 0)
    for aula in model.Aulas for materia in model.Materias
})  # Compatibilidad entre aulas y materias (0 o 1)
# Relación entre profesores y las materias que pueden enseñar
model.Relacion_Profesor_Materia = pyo.Param(model.Profesores, model.Materias, initialize={
    (profe_id, materia_id): 1 if materia_id in datos['Materias_por_Profesor'][profe_id] else 0
    for profe_id in model.Profesores for materia_id in model.Materias
})



In [11]:
# Ver conjuntos
print("Conjuntos:")
print("Profesores:", list(model.Profesores))
print("Materias:", list(model.Materias))
print("Aulas:", list(model.Aulas))
print("Bloques Horarios:", list(model.BloquesHorarios))



# Ver parámetros
print("\nParámetros:")
print("Horas_Semana_Materia:", {m: model.Horas_Semana_Materia[m] for m in model.Materias})
print("Tipo_Clase:", {m: model.Tipo_Clase[m] for m in model.Materias})
print("Tipo_Aula:", {a: model.Tipo_Aula[a] for a in model.Aulas})
print("Compatibilidad_Aula_Materia:", {
    (a, m): model.Compatibilidad_Aula_Materia[a, m] for a in model.Aulas for m in model.Materias
})
print("Relacion_Profesor_Materia:", {
    (p, m): model.Relacion_Profesor_Materia[p, m] for p in model.Profesores for m in model.Materias
})



Conjuntos:
Profesores: [100002999, 100004288, 100012622, 100014033, 100020344, 100054211, 100065344, 100104677, 100104866, 100124799, 100145955, 100171599, 100178366, 100180788, 100209699, 100210066, 100210100, 100226555, 100233866, 100238800, 100239655, 100240399, 100253488, 100260799, 100260822, 100267677, 100268777, 100269800, 100274099, 100274177]
Materias: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Aulas: [(101, 2), (101, 3), (102, 1), (102, 2), (102, 5), (102, 6), (102, 7), (103, 2), (103, 3), (103, 5), (103, 6), (103, 7), (104, 3), (104, 5), (104, 6), (104, 7), (105, 2), (105, 4), (105, 5), (105, 6), (105, 7), (106, 4), (106, 5), (106, 6), (106, 7), (107, 4), (107, 5), (107, 6), (107, 7), (108, 4)]
Bloques Horarios: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 1), (3, 2), (3, 3), (3, 4), (3,

In [26]:
# Variables

# Asignación concreta profesor-materia-aula-horario
model.X = pyo.Var(model.Profesores, model.Materias, model.Aulas, model.BloquesHorarios, domain=pyo.Binary)

# Variables generales: profesor puede enseñar materia (relación)
model.Y = pyo.Var(model.Profesores, model.Materias, domain=pyo.Binary)

# Variables generales: aula asignada a materia
model.Z = pyo.Var(model.Aulas, model.Materias, domain=pyo.Binary)


'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [27]:
if hasattr(model, 'Restriccion_Horas_Semanales'):
    model.del_component(model.Restriccion_Horas_Semanales)

# Asignar exactamente las horas semanales requeridas por cada materia
def horas_semana_materia_rule(model, m):
    return sum(
        model.X[p, m, a, bh]
        for p in model.Profesores
        for a in model.Aulas
        for bh in model.BloquesHorarios
    ) == model.Horas_Semana_Materia[m]

model.Restriccion_Horas_Semanales = pyo.Constraint(model.Materias, rule=horas_semana_materia_rule)

In [28]:
if hasattr(model, 'Restriccion_Profesor_Unico'):
    model.del_component(model.Restriccion_Profesor_Unico)

# No asignar un profesor a más de una materia en un bloque horario
def no_solapamiento_profesor_rule(model, p, dia, hora):
    return sum(
        model.X[p, m, a, (dia, hora)]
        for m in model.Materias
        for a in model.Aulas
    ) <= 1

model.Restriccion_Profesor_Unico = pyo.Constraint(
    model.Profesores, datos['Dia'], datos['Hora'], 
    rule=no_solapamiento_profesor_rule
)

In [29]:
if hasattr(model, 'Restriccion_Aula_Unica'):
    model.del_component(model.Restriccion_Aula_Unica)


# No asignar un aula a más de una materia en un bloque horario
def no_solapamiento_aula_rule(model, id_aula, id_edificio, dia, hora):
    aula_edificio = (id_aula, id_edificio)
    return sum(
        model.X[p, m, aula_edificio, (dia, hora)]
        for p in model.Profesores
        for m in model.Materias
    ) <= 1

model.Restriccion_Aula_Unica = pyo.Constraint(
    model.Aulas, datos['Dia'], datos['Hora'],
    rule=no_solapamiento_aula_rule
)


In [30]:
if hasattr(model, 'Restriccion_Profesor_Compatible'):
    model.del_component(model.Restriccion_Profesor_Compatible)

# Asignar un profesor a una materia solo si el profesor puede enseñar esa materia
def profesor_puede_ensenar_rule(model, p, m, id_aula, id_edificio, dia, hora):
    aula = (id_aula, id_edificio)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Relacion_Profesor_Materia[p, m]

model.Restriccion_Profesor_Compatible = pyo.Constraint(
    model.Profesores, model.Materias, model.Aulas, model.BloquesHorarios,
    rule=profesor_puede_ensenar_rule
)


In [31]:
if hasattr(model, 'Restriccion_Aula_Compatible'):
    model.del_component(model.Restriccion_Aula_Compatible)

# Asignar un aula a una materia solo si el tipo de aula es compatible con el tipo de materia
def aula_compatible_materia_rule(model, p, m, id_aula, id_edificio, dia, hora):
    aula = (id_aula, id_edificio)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Compatibilidad_Aula_Materia[aula, m]

model.Restriccion_Aula_Compatible = pyo.Constraint(
    model.Profesores, model.Materias, model.Aulas, model.BloquesHorarios,
    rule=aula_compatible_materia_rule
)

In [34]:
# %% Restricciones que vinculan variables combinadas con variables generales

if hasattr(model, 'Relacion_X_Y'):
    model.del_component(model.Relacion_X_Y)

def relacion_X_Y_rule(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Y[p, m]

model.Relacion_X_Y = pyo.Constraint(
    model.Profesores,
    model.Materias,
    model.Aulas,
    model.BloquesHorarios,
    rule=relacion_X_Y_rule
)

if hasattr(model, 'Relacion_X_Z'):
    model.del_component(model.Relacion_X_Z)

def relacion_X_Z_rule(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Z[aula, m]

model.Relacion_X_Z = pyo.Constraint(
    model.Profesores,
    model.Materias,
    model.Aulas,
    model.BloquesHorarios,
    rule=relacion_X_Z_rule
)


In [35]:
# Restricciones adicionales para asegurar que cada materia tenga asignado al menos un profesor y un aula

if hasattr(model, 'Materia_Tiene_Profesor'):
    model.del_component(model.Materia_Tiene_Profesor)

def materia_tiene_profesor_rule(model, m):
    return sum(model.Y[p, m] for p in model.Profesores) >= 1

model.Materia_Tiene_Profesor = pyo.Constraint(model.Materias, rule=materia_tiene_profesor_rule)

if hasattr(model, 'Materia_Tiene_Aula'):
    model.del_component(model.Materia_Tiene_Aula)

def materia_tiene_aula_rule(model, m):
    return sum(model.Z[a, m] for a in model.Aulas) >= 1

model.Materia_Tiene_Aula = pyo.Constraint(model.Materias, rule=materia_tiene_aula_rule)

In [ ]:
# Objetivo compuesto: minimizar el número de profesores, aulas y maximizar el uso horario usando una funcion objetivo ponderada

peso_profesores = 1.0      # Minimizar profesores asignados
peso_aulas = 1.0           # Minimizar aulas utilizadas
peso_asignaciones = -0.1   # Maximizar uso horario (negativo porque estamos minimizando)

def objetivo_compuesto_rule(model):
    total_profesores = sum(model.Y[p, m] for p in model.Profesores for m in model.Materias)
    total_aulas = sum(model.Z[a, m] for a in model.Aulas for m in model.Materias)
    total_asignaciones = sum(model.X[p, m, a, bh]
                              for p in model.Profesores
                              for m in model.Materias
                              for a in model.Aulas
                              for bh in model.BloquesHorarios)

    return (peso_profesores * total_profesores +
            peso_aulas * total_aulas +
            peso_asignaciones * total_asignaciones)

model.Objetivo = pyo.Objective(rule=objetivo_compuesto_rule, sense=pyo.minimize)


In [37]:

def resolver_modelo(model, mipgap=0.02, timelimit=120, threads=4, emphasis=1):
    from pyomo.opt import SolverFactory, TerminationCondition

    solver = SolverFactory('cplex')

    if not solver.available():
        raise RuntimeError("CPLEX no está disponible. Verifica la instalación o el PATH.")

    resultado = solver.solve(model, tee=True, options={
        'mipgap': mipgap,         # porcentaje de tolerancia al óptimo
        'timelimit': timelimit,   # tiempo máximo en segundos
        'threads': threads,       # número de núcleos
        'emphasis': emphasis,     # 1 = prioriza factibilidad
        'mipdisplay': 2           # muestra progreso intermedio
    })

    # Verificar si la solución fue óptima o factible
    condicion = resultado.solver.termination_condition
    if condicion in [TerminationCondition.optimal, TerminationCondition.feasible]:
        print(f"Solución encontrada: {condicion}")
    else:
        print(f"Problema durante la resolución: {condicion}")
        print("Revisa el modelo o ajusta parámetros del solver.")

    return resultado


In [38]:
def mostrar_bloque(bloque):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
    dia, hora = bloque
    dia_str = dias[dia - 1]
    hora_inicio = 7 + hora  # porque en tu modelo hora=1 corresponde a 8 am, aquí sumo 7 para que sea 8
    hora_fin = hora_inicio + 1
    return f"{dia_str} {hora_inicio:02d}:00 - {hora_fin:02d}:00"


def mostrar_asignacion(model):
    print("\nAsignaciones de profesores, materias, aulas y horarios:")
    for p in model.Profesores:
        for m in model.Materias:
            for a in model.Aulas:
                for bh in model.BloquesHorarios:
                    if pyo.value(model.X[p, m, a, bh]) > 0.5:
                        print(f"Profesor: {datos['nombres_profesor'][p]} | Materia: {datos['nombres_materia'][m]} | Aula: {a} | Horario: {mostrar_bloque(bh)}")


In [ ]:
# Resolver el modelo
resultados = resolver_modelo(model)

# Verificar si encontró solución
if (resultados.solver.status == pyo.SolverStatus.ok and
    resultados.solver.termination_condition == TerminationCondition.optimal):

    print("Solución óptima encontrada.\n")
    mostrar_asignacion(model)

else:
    print("No se encontró solución óptima.")
    print("Estado:", resultados.solver.status)
    print("Condición de terminación:", resultados.solver.termination_condition)


In [ ]:
#Objetivo Lexicográfica: Primero minimizar el número de profesores, luego el número de aulas y finalmente maximizar el uso horario

from pyomo.opt import SolverFactory

opt = SolverFactory("glpk")  # O el solver que uses

# ========================
# Paso 1: Minimizar profesores asignados
def objetivo_min_profesores_rule(model):
    return sum(model.Y[p, m] for p in model.Profesores for m in model.Materias)
model.Objetivo = pyo.Objective(rule=objetivo_min_profesores_rule, sense=pyo.minimize)

result1 = opt.solve(model)
model.pprint()

# Guardar valor óptimo
profesores_min = sum(pyo.value(model.Y[p, m]) for p in model.Profesores for m in model.Materias)

# Agregar restricción para mantener este mínimo en el siguiente paso
model.del_component(model.Objetivo)
model.Restriccion_Profesores_Fijos = pyo.Constraint(expr=
    sum(model.Y[p, m] for p in model.Profesores for m in model.Materias) <= profesores_min
)

# ========================
# Paso 2: Minimizar aulas asignadas
def objetivo_min_aulas_rule(model):
    return sum(model.Z[a, m] for a in model.Aulas for m in model.Materias)
model.Objetivo = pyo.Objective(rule=objetivo_min_aulas_rule, sense=pyo.minimize)

result2 = opt.solve(model)

aulas_min = sum(pyo.value(model.Z[a, m]) for a in model.Aulas for m in model.Materias)

# Fijar también el número de aulas usadas
model.del_component(model.Objetivo)
model.Restriccion_Aulas_Fijas = pyo.Constraint(expr=
    sum(model.Z[a, m] for a in model.Aulas for m in model.Materias) <= aulas_min
)

# ========================
# Paso 3: Maximizar asignaciones horarias (X)
def objetivo_max_asignaciones_rule(model):
    return sum(model.X[p, m, a, bh]
               for p in model.Profesores
               for m in model.Materias
               for a in model.Aulas
               for bh in model.BloquesHorarios)
model.Objetivo = pyo.Objective(rule=objetivo_max_asignaciones_rule, sense=pyo.maximize)

result3 = opt.solve(model)
